In [1]:
#Import required libraries
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import json 


!pip install geocoder
!pip install geopy
from geopy.geocoders import Nominatim 
import geocoder
import geopy

import requests  

from pandas.io.json import json_normalize 

from bs4 import BeautifulSoup

import matplotlib.cm as cm
import matplotlib.colors as colors


!pip install sklearn
from sklearn.cluster import KMeans

!pip install folium
import folium 

print('Libraries imported.')

Libraries imported.


In [27]:
#For getting neighbourhood current population,I used "https://www.talent-berlin.de/en/living/districts-neighborhoods"

from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

url=requests.get("https://www.talent-berlin.de/en/living/districts-neighborhoods").text
soup=BeautifulSoup(url,"lxml")

table=soup.find_all("tr")
headers = ["Borough","Area in ha","Population"]
     
rows = []
for row in table:
    td = row.find_all('td')
    row = [row.text for row in td]
    rows.append(row)

with open('neighbourhood.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(row for row in rows if row)

df1=pd.read_csv("neighbourhood.csv",encoding= 'unicode_escape')


In [28]:
#Current Berlin population by borough
df1

,Borough,Area in ha,Population
0,Mitte,"3,947","371,407"
1,Pankow,"10,301","397,406"
2,Charlottenburg-Wilmersdorf,"6,472","336,249"
3,Friedrichshain-Kreuzberg,"2,016","281,323"
4,Spandau,"9,191","239,942"
5,Reinickendorf,"8,946","261,919"
6,Lichtenberg,"5,229","283,121"
7,Marzahn-Hellersdorf,"6,174","262,015"
8,Treptow-Köpenick,"16,842","259,524"
9,Neukölln,"4,493","327,522"


In [29]:
# Geographical Coordinates of Neighborhoods


def get_latlng(borough):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Berlin, Germany'.format(borough))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(borough) for borough in df1["Borough"].tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
df1['Latitude'] = df_coords['Latitude']
df1['Longitude'] = df_coords['Longitude']


# check the neighborhoods and the coordinates
print(df1.shape)
df1.head()

(12, 5)


,Borough,Area in ha,Population,Latitude,Longitude
0,Mitte,"3,947","371,407",52.52119,13.42414
1,Pankow,"10,301","397,406",52.56925,13.40248
2,Charlottenburg-Wilmersdorf,"6,472","336,249",52.49962,13.32316
3,Friedrichshain-Kreuzberg,"2,016","281,323",52.50061,13.41863
4,Spandau,"9,191","239,942",52.53487,13.20216


In [30]:
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent = "myapp1")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Berlin is 52.5170365, 13.3888599.


In [31]:
# Create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df1['Latitude'], df1['Longitude'], df1['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_berlin)  
    
map_berlin

## Using Foursquare API

In [32]:
#Now we will try to get venues by each borough of Berlin

In [33]:
CLIENT_ID = 'YP03FATPA3WT3GI4K5D5EC2XOTUE3YKFYULHSR1DBJSIKK5M' # your Foursquare ID
CLIENT_SECRET = 'FQTXNXDDJYMZVYAUHCKY5TVVF1WZPJKLDMI411XXAGYOHV2P' # your Foursquare Secret
ACCESS_TOKEN = '42GH2CXALXEM5R1VEXYBCUVJETZ31E135YWUNA1DSPDPFMTT' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YP03FATPA3WT3GI4K5D5EC2XOTUE3YKFYULHSR1DBJSIKK5M
CLIENT_SECRET:FQTXNXDDJYMZVYAUHCKY5TVVF1WZPJKLDMI411XXAGYOHV2P


In [34]:
radius = 1500
LIMIT = 50

venues = []
for lat, long, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [90]:
# Convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Borough', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(519, 7)


,Borough,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Mitte,52.52119,13.42414,Ristorante a Mano,52.518772,13.427652,Italian Restaurant
1,Mitte,52.52119,13.42414,Kino International,52.520363,13.422849,Indie Movie Theater
2,Mitte,52.52119,13.42414,Märchenbrunnen,52.527999,13.426752,Fountain
3,Mitte,52.52119,13.42414,Freiluftkino Friedrichshain,52.525380,13.435410,Indie Movie Theater
4,Mitte,52.52119,13.42414,Flying Tiger,52.522437,13.412612,Gift Shop


In [91]:
venues_df.groupby(["Borough"]).count()
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 160 uniques categories.


160 different  venues categories are available in Berlin.We are going to focus on Italian Restaurants.

In [92]:
#### Analyse each neighborhood
# one hot encoding
onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Boroughs'] = venues_df['Borough'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape)

grouped = onehot.groupby(["Boroughs"]).mean().reset_index()

print(grouped.shape)
grouped

(519, 161)
(12, 161)


,Boroughs,African Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,Bagel Shop,Bakery,Bar,Bathing Area,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Canal,Candy Store,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Cosmetics Shop,Costume Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hostel,Hotel,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Kumpir Restaurant,Kurdish Restaurant,Lake,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Organic Grocery,Outdoor Supply Store,Palace,Park,Pastry Shop,Peruvian Restaurant,Pet Store,Photography Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Record Shop,Recreation Center,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Spa,Spanish Restaurant,Squash Court,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Syrian Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Tour Provider,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,Charlottenburg-Wilmersdorf,0.00,0.00,0.00,0.02,0.00,0.020000,0.00,0.00,0.06,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.000000,0.00,0.00,0.02,0.00,0.000000,0.060000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.02,0.000000,0.00,0.000000,0.00,0.020000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.02,0.00,0.00,0.00,0.00,0.040000,0.00,0.02,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.160000,0.000000,0.000000,0.00,0.000000,0.04,0.02,0.04,0.00,0.02,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.02,0.00,0.000000,0.00,0.02,0.00,0.00,0.02,0.000000,0.00,0.000000,0.00,0.00,0.04,0.00,0.00,0.00,0.000000,0.020000,0.00,0.000000,0.00,0.000000,0.00,0.02,0.000000,0.02,0.00,0.00,0.00,0.04,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.000000,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.02,0.00,0.04,0.00
1,Friedrichshain-Kreuzberg,0.02,0.00,0.02,0.00,0.02,0.000000,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.02,0.000000,0.020000,0.02,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.02,0.12,0.02,0.00,0.00,0.00,0.00,0.02,0.000000,0.02,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.02,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00,0.02,0.020000,0.00,0.02,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.04,0.000000,0.00,0.00,0.00,0.00,0.02,0.02,0.02,0.000000,0.000000,0.000000,0.00,0.02,0.000000,0.02,0.00,0.000000,0.00,0.02,0.00,0.00,0.00,0.020000,0.02,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.02,0.000000,0.02,0.020000,0.00,0.00,0.000000,0.00,0.00,0.04,0

We decreased  520 observers to 12 by groupby.

In [93]:
len(grouped[grouped["Italian Restaurant"] > 0])
berlin_rest = grouped[["Boroughs","Italian Restaurant","Bar"]]
berlin_rest

,Boroughs,Italian Restaurant,Bar
0,Charlottenburg-Wilmersdorf,0.04,0.02
1,Friedrichshain-Kreuzberg,0.00,0.04
2,Lichtenberg,0.04,0.02
3,Marzahn-Hellersdorf,0.00,0.00
4,Mitte,0.04,0.00
5,Neukölln,0.00,0.02
6,Pankow,0.02,0.02
7,Reinickendorf,0.00,0.00
8,Spandau,0.00,0.00
9,Steglitz-Zehlendorf,0.08,0.00


## Clustering

In [94]:
#Now it is time to divide Berlin in clusters
kclusters = 4

berlin_clustering = berlin_rest.drop(["Boroughs"], 1)

# run k-means clustering
kmeans = KMeans(init="k-means++", n_clusters=kclusters, n_init=12).fit(berlin_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 1, 0, 1, 2, 2, 0, 0, 3])

In [95]:
merged = berlin_rest.copy()

# add clustering labels
merged["Category"] = kmeans.labels_
merged.rename(columns={"Boroughs": "Borough"}, inplace=True)
merged.head()

,Borough,Italian Restaurant,Bar,Category
0,Charlottenburg-Wilmersdorf,0.04,0.02,1
1,Friedrichshain-Kreuzberg,0.00,0.04,2
2,Lichtenberg,0.04,0.02,1
3,Marzahn-Hellersdorf,0.00,0.00,0
4,Mitte,0.04,0.00,1


In [96]:
dfmerged = merged.merge(df1)
dfmerged.head()

#Sort
dfmerged.sort_values(["Italian Restaurant"], inplace=True, ascending=True)
dfmerged

,Borough,Italian Restaurant,Bar,Category,Area in ha,Population,Latitude,Longitude
1,Friedrichshain-Kreuzberg,0.00,0.04,2,"2,016","281,323",52.500610,13.418630
3,Marzahn-Hellersdorf,0.00,0.00,0,"6,174","262,015",52.516670,13.583330
5,Neukölln,0.00,0.02,2,"4,493","327,522",52.480770,13.435410
7,Reinickendorf,0.00,0.00,0,"8,946","261,919",52.575450,13.349700
8,Spandau,0.00,0.00,0,"9,191","239,942",52.534870,13.202160
11,Treptow-Köpenick,0.00,0.00,0,"16,842","259,524",52.433330,13.600000
6,Pankow,0.02,0.02,2,"10,301","397,406",52.569250,13.402480
0,Charlottenburg-Wilmersdorf,0.04,0.02,1,"6,472","336,249",52.499620,13.323160
2,Lichtenberg,0.04,0.02,1,"5,229","283,121",52.516180,13.480870
4,Mitte,0.04,0.00,1,"3,947","371,407",52.521190,13.424140


In [97]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfmerged['Latitude'], dfmerged['Longitude'], dfmerged['Borough'], dfmerged['Category']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [98]:
merged.loc[merged['Category'] == 0]

,Borough,Italian Restaurant,Bar,Category
3,Marzahn-Hellersdorf,0.0,0.0,0
7,Reinickendorf,0.0,0.0,0
8,Spandau,0.0,0.0,0
11,Treptow-Köpenick,0.0,0.0,0


In [99]:
merged.loc[merged['Category'] == 1]

,Borough,Italian Restaurant,Bar,Category
0,Charlottenburg-Wilmersdorf,0.04,0.02,1
2,Lichtenberg,0.04,0.02,1
4,Mitte,0.04,0.00,1
10,Tempelhof-Schöneberg,0.04,0.00,1


In [100]:
merged.loc[merged['Category'] == 2]

,Borough,Italian Restaurant,Bar,Category
1,Friedrichshain-Kreuzberg,0.00,0.04,2
5,Neukölln,0.00,0.02,2
6,Pankow,0.02,0.02,2


In [101]:
merged.loc[merged['Category'] == 3]

,Borough,Italian Restaurant,Bar,Category
9,Steglitz-Zehlendorf,0.08,0.0,3


### According to our Italian Restaurant number in the clusters ,the best category will be category 0. I will explain it in report.However,in here i want to take a second paremeter, beside Italian numbers, that includes which boroughs have much Italian immigrants. 

In [102]:
import csv

website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_Berlin').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
    td = row.find_all('td')
    row = [row.text for row in td]
    rows.append(row)


with open('wiki.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(row for row in rows if row)
    
df2=pd.read_csv('wiki.csv',encoding= 'unicode_escape')
df2



,Borough,Population 2010,Area in km²,Largest Non-German ethnic groups\r\n
0,Mitte,"332,100",39.47,"Turks, Arabs, Kurds, Asians, Western Europeans..."
1,Friedrichshain-Kreuzberg,"268,831",20.16,"Turks, Arabs, Africans, Kurds, Chinese\r\n"
2,Pankow,"368,956",103.01,"Poles, Italians, French, Americans, Vietnamese..."
3,Charlottenburg-Wilmersdorf,"320,014",64.72,"Turks, Africans, Russians, Arabs\r\n"
4,Spandau,"225,420",91.91,"Turks, Africans, Russians, Arabs\r\n"
5,Steglitz-Zehlendorf,"293,989",102.50,"Poles, Turks, Croats, Serbs, Koreans\r\n"
6,Tempelhof-Schöneberg,"335,060",53.09,"Turks, Croats, Serbs, Koreans, Africans\r\n"
7,Neukölln,"310,283",44.93,"Arabs, Turks, Kurds, Russians, Africans, Poles..."
8,Treptow-Köpenick,"241,335",168.42,"Russians, Poles, Ukrainians, Vietnamese\r\n"
9,Marzahn-Hellersdorf,"248,264",61.74,"Russians, Vietnamese, Eastern Europeans\r\n"


In [103]:
df2 = df2[df2['Largest Non-German ethnic groups\r\n'].str.contains('Italian',regex=False)]
df2.columns

Index(['Borough', 'Population 2010', 'Area in km²',
       'Largest Non-German ethnic groups\r\n'],
      dtype='object')

In [104]:
df2.drop(['Population 2010', 'Area in km²'],axis=1, inplace=True)
df2

C:\Users\OBO\anaconda3\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Borough,Largest Non-German ethnic groups\r\n
2,Pankow,"Poles, Italians, French, Americans, Vietnamese..."
11,Reinickendorf,"Turks, Poles, Serbs, Croats, Arabs, Italians\r\n"


It can be seen, of course according to Wikipedia, 2 Boroughs have Italian living groups.

### I merged the tables, one of them shows Italian immigrants boroughs and the second one shows Italian restuarants number

In [105]:
dfmerged = dfmerged.merge(df2)
dfmerged

,Borough,Italian Restaurant,Bar,Category,Area in ha,Population,Latitude,Longitude,Largest Non-German ethnic groups\r\n
0,Reinickendorf,0.00,0.00,0,"8,946","261,919",52.57545,13.34970,"Turks, Poles, Serbs, Croats, Arabs, Italians\r\n"
1,Pankow,0.02,0.02,2,"10,301","397,406",52.56925,13.40248,"Poles, Italians, French, Americans, Vietnamese..."


In [106]:
dfmerged.rename(columns = {'Largest Non-German ethnic groups\r\n': 'Largest Non-German ethnic groups'}, inplace = True)

In [107]:
dfmerged

,Borough,Italian Restaurant,Bar,Category,Area in ha,Population,Latitude,Longitude,Largest Non-German ethnic groups
0,Reinickendorf,0.00,0.00,0,"8,946","261,919",52.57545,13.34970,"Turks, Poles, Serbs, Croats, Arabs, Italians\r\n"
1,Pankow,0.02,0.02,2,"10,301","397,406",52.56925,13.40248,"Poles, Italians, French, Americans, Vietnamese..."


In [108]:
dfmerged.replace(to_replace ="Turks, Poles, Serbs, Croats, Arabs, Italians\r\n",
                 value ="Turks, Poles, Serbs, Croats, Arabs, Italians",inplace=True)
dfmerged

,Borough,Italian Restaurant,Bar,Category,Area in ha,Population,Latitude,Longitude,Largest Non-German ethnic groups
0,Reinickendorf,0.00,0.00,0,"8,946","261,919",52.57545,13.34970,"Turks, Poles, Serbs, Croats, Arabs, Italians"
1,Pankow,0.02,0.02,2,"10,301","397,406",52.56925,13.40248,"Poles, Italians, French, Americans, Vietnamese..."


#### After merging the table includes Italian restuarant numbers and  the table includes more  Italian immigrants living boroughs, the results say us there are 2 options.I am going to pick Pankow up and the explanation will be in the report.